In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.model_selection import GroupKFold

In [2]:
SEED = 2022
FOLDS = 5
INPUT_PATH = Path("../input/isic-2024-challenge/")
OUTPUT_FILENAME = "folds.csv"

In [3]:
train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False);print(f"Train data size: {train_metadata.shape}")

Train data size: (401059, 55)


In [4]:
id_column = "isic_id"
target_column = "target"
group_column = "patient_id"
fold_column = "fold"
final_target_column = "final_target"

folds = np.zeros(train_metadata.shape[0], dtype=int)

gkf = GroupKFold(n_splits=FOLDS)
for fold_idx, (_, val_idx) in enumerate(gkf.split(train_metadata, train_metadata[target_column], train_metadata[group_column])):
    folds[val_idx] = fold_idx + 1

train_metadata[fold_column] = folds

In [5]:
train_metadata[final_target_column] = train_metadata[target_column]

In [6]:
columns_to_write = [id_column, group_column, final_target_column, fold_column]
train_metadata[columns_to_write].head(n=10)

,isic_id,patient_id,final_target,fold
0,ISIC_0015670,IP_1235828,0,4
1,ISIC_0015845,IP_8170065,0,1
2,ISIC_0015864,IP_6724798,0,5
3,ISIC_0015902,IP_4111386,0,2
4,ISIC_0024200,IP_8313778,0,1
5,ISIC_0035502,IP_3026693,0,5
6,ISIC_0051648,IP_0218255,0,1
7,ISIC_0051665,IP_7734648,0,2
8,ISIC_0051710,IP_1307115,0,2
9,ISIC_0051758,IP_2180091,0,5


In [7]:
train_metadata[final_target_column].value_counts(normalize=True)

final_target
0    0.99902
1    0.00098
Name: proportion, dtype: float64

In [8]:
train_metadata[columns_to_write].to_csv(OUTPUT_FILENAME, index=False)